# Simulation 3

In [ ]:
"""
Simulating the WTSN setting

Authors: Milind Kumar Vaddiraju, ChatGPT, Copilot
"""

# Necessary imports
import copy
from datetime import datetime
import json
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import sys

from network_classes import *
from utils import *


In [ ]:
# Parameters affecting how a packet is served: essentially MCS and latency from the Excel sheet
# TODO: integrate MCS usage into the UE instead of having it outside
# TODO: Create a simple CSV file of this


# TODO: Remove the 67us from this that contains backoff 
parameters = {
    "setting 0": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 0,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 1,
        "delivery_latency": 591.8
    },
    "setting 1": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 0,
        "PER": 0,
        "payload_size": 964,
        "aggregation": 1,
        "delivery_latency": 1470.2
    },
    "setting 2": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 0,
        "PER": 0,
        "payload_size": 2464,
        "aggregation": 1,
        "delivery_latency": 2953.4
    },
    "setting 3": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 1,
        "delivery_latency": 534.2
    },
    "setting 4": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 964,
        "aggregation": 1,
        "delivery_latency": 980.6
    },
    "setting 5": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 2464,
        "aggregation": 1,
        "delivery_latency": 1715
    },
    "setting 6": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 1,
        "delivery_latency": 519.8
    },
    "setting 7": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 964,
        "aggregation": 1,
        "delivery_latency": 807.8
    },
    "setting 8": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 2464,
        "aggregation": 1,
        "delivery_latency": 1311.8
    },
    "setting 9": {
        "SNR": 20,
        "Bandwidth": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 10,
        "delivery_latency": 980.6
    },
    "setting 10": {
        "SNR": 35,
        "Bandwidth": 20,
        "MCS": 8,
        "PER": 0.1170412, # PER corresponding to PSDU size 1000 as aggregation of 10 x 64 = 1000 B
        "payload_size": 64,
        "aggregation": 10,
        "delivery_latency": 563 
    },
    "setting 11": {
        "SNR": 38,
        "Bandwidth": 20,
        "MCS": 6,
        "PER": 0, # PER corresponding to PSDU size 1000 as aggregation of 10 x 64 = 1000 B
        "payload_size": 64,
        "aggregation": 10,
        "delivery_latency": 591.8 
    },
}



In [ ]:
# Set the simulation parameters

results_directory_simulation = "./results/simulation_3/"

setting_reserved = "setting 11"
setting_contention = "setting 11"
payload_size = {"reserved": parameters[setting_reserved]["payload_size"]*parameters[setting_reserved]["aggregation"], 
                "contention": parameters[setting_contention]["payload_size"]*parameters[setting_contention]["aggregation"]}
delivery_latency = {"reserved": parameters[setting_reserved]["delivery_latency"],
                    "contention": parameters[setting_contention]["delivery_latency"]}
PER = {"reserved":  parameters[setting_reserved]["PER"], 
       "contention":  parameters[setting_contention]["PER"]}



num_UEs = 10
UE_names = ["UE" + str(i) for i in range(num_UEs)]
num_packets_per_ue = None  # Number of packets per UE for the whole period
packet_sizes = [parameters[setting_reserved]["payload_size"]] # TODO: Both have same packet size, but what if they don't?
priorities = [1]
# lambda_range = np.logspace(-4.5, -2.3, 10)
lambda_range = np.concatenate(np.logspace(-4.5, -3, 10), np.logspace(-3, -2.2, 5))
lambda_original = copy.deepcopy(lambda_range)
UE_arrival = ["Poisson"]*num_UEs
UE_serve_mode = ["Mode 2"]*num_UEs
num_iterations_arrival = 10


## Schedule parameters for reserved base schedule
num_slots_per_UE = 200
num_slots = num_slots_per_UE*num_UEs
start_offset = 10 # microseconds
end_time = start_offset
slot_duration = delivery_latency["reserved"] + 200 # microseconds


# Network properties
# Obtained from the sheet
wifi_slot_time = 9 # microseconds
DIFS = 34 # microseconds



# Plot information
percentile_to_plot = 99
num_iterations_contention = 5
mode_contention = "Mode 3" 
advance_time = 10 # microseconds
debug_mode = False


In [ ]:
# Create a schedule, UEs and serve the packets


# TODO: move the knowledge of how many packets there are to this part of the code
# instead of keeping it in the UE class

# Create a schedule
slots = {}
start_time = start_offset
for i in range(num_slots):
    slots[i] = Slot(i, start_time, start_time + slot_duration, "reserved", [UE_names[i%num_UEs]])
    start_time += slot_duration
schedule_reserved = Schedule(start_offset, start_time, num_slots, slots)

end_time = start_time # Due to variable use while saving experiment parameters

slots_temp = {}
slots_temp[0] = Slot(0, start_offset, end_time, "contention", UE_names)
schedule_contention = Schedule(start_offset, end_time, 1, slots_temp)


# print(schedule_reserved)
print(schedule_contention)

results_per_lambda = {}
results_per_lambda_contention = {}

count = 0

for lambda_value in lambda_range:
    
    print("\n###### Lambda value: " + str(lambda_value), ", Count: " + str(count), "######")
    count = count + 1
    
    result = {}
    results_per_lambda_per_iteration_contention = {}
    for num_arrival_iteration in range(num_iterations_arrival):
        print("\nArrival iteration: " + str(num_arrival_iteration))
        # Create UEs and packets
        
        UEs = {}
            
        for i in range(num_UEs): 
            # TODO: Move the UE creation parameters to the cell above?
            UE_temp = UE(i, {1: 0, 2: 1}, UE_arrival[i], UE_serve_mode[i],  num_packets_per_ue)
            UE_temp.set_poisson_lambda(lambda_value)
            UE_temp.generate_packets(schedule_reserved, packet_sizes, priorities)
            UEs[UE_names[i]] = UE_temp
        
        UEs_contention = copy.deepcopy(UEs)

        # for i in UEs:
        #     print(UEs[i])

        # Serve the packets
        # Serving only one UE to reduce time taken to run code
        for i in range(num_UEs):
            UEs[UE_names[i]].serve_packets(schedule_reserved, 
                                        payload_size=payload_size["reserved"], 
                                        delivery_latency=delivery_latency["reserved"],
                                        PER=PER["reserved"])

        # for i in UEs:
        #     print(UEs[i])
            

        
        result[num_arrival_iteration] = UEs
        
        

        # TODO: Check that the delivery times are always in ascending order
        # TODO: check that the arrival times are always in ascending order

        # TODO: Make this more general i.e handle packet statuses directly instead of opearting under the 
        # restrictions of this simulation
        print("Num packets, reserved: " + str(UEs["UE0"].n_packets))


        # Serve the packets with contention
        results_iteration = {}
        

        for i in range(num_iterations_contention):
            print("Contention iteration: " + str(i))
            UEs_contention_temp = copy.deepcopy(UEs_contention)

            test_network = Network(wifi_slot_time, DIFS, UEs_contention_temp, debug_mode)
            test_network.serve_packets(schedule_contention, mode_contention, 
                                        payload_size = payload_size,
                                        delivery_latency = delivery_latency,
                                        PER = PER,
                                        advance_time = advance_time)
            
            


            results_iteration[i] = UEs_contention_temp 
        # for key in results_iteration:
        #     print("results_iteration " + str(key), results_iteration[key])

        # TODO: Scale to multiple UEs, currently you're extracting the results only for one UE,
        # but you should be extracting the results for all UEs
        

        results_per_lambda_per_iteration_contention[num_arrival_iteration] = results_iteration
    
    results_per_lambda[lambda_value] = result
    results_per_lambda_contention[lambda_value] = results_per_lambda_per_iteration_contention
    



In [ ]:
# Create a results directory folder using results_directory_simulation and the current time
experiment_folder_name = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
results_directory_experiment = os.path.join(results_directory_simulation, experiment_folder_name)
os.makedirs(results_directory_experiment, exist_ok=True)
# Plots: CDF of latencies, percentile latency vs lambda, mean latency vs lambda,
# number of packets not served vs lambda

In [ ]:
results_allUEs_per_lambda_contention = {}
for lambda_value in results_per_lambda_contention:
    print("lambda value: ", lambda_value)

    mean_latencies_across_arrivals = []
    percentile_latencies_across_arrivals = []

    for num_iteration_arrival in results_per_lambda_contention[lambda_value]:
        mean_latencies = []
        percentile_latencies = []
        print("arrival iteration " + str(num_iteration_arrival) + "\n")
        for iteration in results_per_lambda_contention[lambda_value][num_iteration_arrival]:
            latencies = []
            # print("iteration", iteration)
            for ue in results_per_lambda_contention[lambda_value][num_iteration_arrival][iteration]:
                # print("UE: ", ue)
                UE_temp = results_per_lambda_contention[lambda_value][num_iteration_arrival][iteration][ue]
                latencies_UE = UE_temp.obtain_packet_latency()
                latencies_UE = [latency for latency in latencies_UE if latency is not None]
                latencies.extend(latencies_UE)
            print("iteration", iteration)    
            mean_latencies.append(np.mean(latencies))
            percentile_latencies.append(compute_percentile(latencies, percentile_to_plot))
        print("Len(mean_latencies)", len(mean_latencies))
        mean_latencies_across_arrivals.append(np.mean(mean_latencies))
        percentile_latencies_across_arrivals.append(np.mean(percentile_latencies))

    result_temp = {}        
    result_temp["mean_latency"] = np.mean(mean_latencies_across_arrivals)
    result_temp["mean_latency_std"] = np.std(mean_latencies_across_arrivals)
    result_temp["percentile_latency"] = np.mean(percentile_latencies_across_arrivals)
    result_temp["percentile_latency_std"] = np.std(percentile_latencies_across_arrivals)
    results_allUEs_per_lambda_contention[lambda_value] = result_temp


results_allUEs_per_lambda_reserved = {}
for lambda_value in results_per_lambda:
    print("lambda value: ", lambda_value)

    mean_latencies_across_arrivals = []
    percentile_latencies_across_arrivals = []

    for num_iteration_arrival in results_per_lambda[lambda_value]:
        
        latencies = []
        print("arrival iteration", num_iteration_arrival)
        for ue in results_per_lambda[lambda_value][num_iteration_arrival]:
            # print("UE: ", ue)
            UE_temp = results_per_lambda[lambda_value][num_iteration_arrival][ue]
            latencies_UE = UE_temp.obtain_packet_latency()
            latencies_UE = [latency for latency in latencies_UE if latency is not None]
            latencies.extend(latencies_UE)


        mean_latencies_across_arrivals.append(np.mean(latencies))
        percentile_latencies_across_arrivals.append(compute_percentile(latencies, percentile_to_plot))
        print("Len(mean_latencies)", len(mean_latencies_across_arrivals))

    print("mean_latencies_across_arrivals", mean_latencies_across_arrivals)

    result_temp = {}        
    result_temp["mean_latency"] = np.mean(mean_latencies_across_arrivals)
    result_temp["mean_latency_std"] = np.std(mean_latencies_across_arrivals)
    result_temp["percentile_latency"] = np.mean(percentile_latencies_across_arrivals)
    result_temp["percentile_latency_std"] = np.std(percentile_latencies_across_arrivals)
    results_allUEs_per_lambda_reserved[lambda_value] = result_temp

# lambda_range = lambda_range[10:]





In [ ]:
# Save the parameters and the results of the experiment to a file

experiment_parameters = {
    "setting_reserved": parameters[setting_reserved],
    "setting_contention": parameters[setting_contention],
    "num_UEs": num_UEs,
    "num_packets_per_ue": num_packets_per_ue,
    "packet_sizes": packet_sizes,
    "priorities": priorities,
    "UE_arrival": UE_arrival,
    "UE_serve_mode": UE_serve_mode,
    "num_slots_per_UE": num_slots_per_UE,
    "num_slots": num_slots,
    "start_offset": start_offset, # microseconds
    "end_time": end_time,
    "slot_duration": slot_duration,
    "percentile_to_plot": percentile_to_plot,
    "wifi_slot_time": wifi_slot_time,
    "DIFS": DIFS,
    "num_iterations_contention": num_iterations_contention,
    "num_iterations_arrival": num_iterations_arrival,
    "contention_mode": mode_contention,
    "advance_time": advance_time,
    "lambda_range": lambda_range,
}

# Write experiment_parameters_json to a json file with filename experiment_parameters.json

experiment_parameters_json = json.dumps(experiment_parameters, indent=4, cls=NumpyEncoder)
experiment_parameters_json_filename = os.path.join(results_directory_experiment, \
                                                   "experiment_parameters.json")
with open(experiment_parameters_json_filename, "w") as file:
    file.write(experiment_parameters_json)


experiment_parameters_pickle = {
    "schedule_reserved": schedule_reserved,
    "schedule_contention": schedule_contention,
    "results_per_lambda": results_per_lambda,
    "results_per_lambda_contention": results_per_lambda_contention,
    "results_allUEs_per_lambda_reserved": results_per_lambda_per_iteration_contention,
    "results_allUEs_per_lambda_contention": results_allUEs_per_lambda_contention,
    "experiment_parameters": experiment_parameters
}

experiment_parameters_pickle_filename = os.path.join(results_directory_experiment, \
                                                    "experiment_parameters.pkl")

with open(experiment_parameters_pickle_filename, "wb") as file:
    pickle.dump(experiment_parameters_pickle, file)

In [ ]:
lambda_range = lambda_original

lambda_range = lambda_range[:count - 1]

scale = "linear"

# Plot the percentile curve

plt.figure(figsize=(10, 8))
# percentiles = []
# for lambda_value in lambda_range:
#     percentiles.append(results_allUEs_per_lambda_reserved[lambda_value]["percentile_latency"])
# plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time), \
#          percentiles, ".-", label = "reserved")


percentiles = []
percentiles_std = []
for lambda_value in lambda_range:
    percentiles.append(results_allUEs_per_lambda_reserved[lambda_value]["percentile_latency"])
    percentiles_std.append(\
        results_allUEs_per_lambda_reserved[lambda_value]["percentile_latency_std"])
plt.errorbar(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time), \
        percentiles, percentiles_std, label = "reserved", fmt='.-', \
        capsize=3)



percentiles_contention = []
percentiles_contention_std = []
for lambda_value in lambda_range:
    percentiles_contention.append(results_allUEs_per_lambda_contention[lambda_value]["percentile_latency"])
    percentiles_contention_std.append(\
        results_allUEs_per_lambda_contention[lambda_value]["percentile_latency_std"])
plt.errorbar(np.array(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time), \
        percentiles_contention, percentiles_contention_std, label = "contention", fmt='.-', \
        capsize=3)
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda*schedule_duration (us)")
plt.ylabel(str(percentile_to_plot) + "percentile latency (us)")
plt.legend()

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 {percentile_to_plot} percentile latency vs lambda, \n PER = {PER},\n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
         f"delivery_latency: {delivery_latency} us ,\n"
         f"num_slots: {num_slots}, \n"
         f"slot_duration: {slot_duration} us ,\n"
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label
plt.text(0, percentiles[0], str(np.round(percentiles[0],2)), fontsize=12, verticalalignment='bottom')
plt.text(0, percentiles_contention[0], str(np.round(percentiles_contention[0],2)), \
         fontsize=12, verticalalignment='bottom')
plt.tight_layout()

if scale == "log":
        plt.savefig(os.path.join(results_directory_experiment, "percentile_latency_allUEs_log.png"))
elif scale == "linear":
        plt.savefig(os.path.join(results_directory_experiment, "percentile_latency_allUEs_linear.png"))
plt.show()


slope = np.diff(percentiles_contention)/np.diff(lambda_range)
plt.yscale('log')
plt.plot(np.array(lambda_range[1:])*(schedule_contention.end_time - schedule_contention.start_time), slope)

# Plot the mean latency curve

plt.figure(figsize=(10, 8))
# mean_latencies = []
# for lambda_value in lambda_range:
#     mean_latencies.append(results_allUEs_per_lambda_reserved[lambda_value]["mean_latency"])
# plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time),\
#          mean_latencies, ".-", label = "reserved")

mean_latencies = []
mean_latencies_std = []
for lambda_value in lambda_range:
    mean_latencies.append(results_allUEs_per_lambda_reserved[lambda_value]["mean_latency"])
    mean_latencies_std.append(\
        results_allUEs_per_lambda_reserved[lambda_value]["mean_latency_std"])
plt.errorbar(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time), \
        mean_latencies, mean_latencies_std, label = "reserved", fmt='.-', \
        capsize=3)

mean_latencies_contention = []
mean_latencies_contention_std = []
for lambda_value in lambda_range:
    mean_latencies_contention.append(results_allUEs_per_lambda_contention[lambda_value]["mean_latency"])
    mean_latencies_contention_std.append(\
        results_allUEs_per_lambda_contention[lambda_value]["mean_latency_std"])
plt.errorbar(np.array(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time),\
        mean_latencies_contention, mean_latencies_contention_std, label = "contention", fmt='.-', \
        capsize=3)

plt.text(0, mean_latencies[0], str(np.round(mean_latencies[0],2)), fontsize=12, verticalalignment='top')
plt.text(0, mean_latencies_contention[0], str(np.round(mean_latencies_contention[0],2)), \
         fontsize=12, verticalalignment='bottom')

plt.legend()

plt.xlabel("lambda*schedule_duration")
plt.ylabel("Mean latency (us)")

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 mean latency vs lambda, \n PER = {PER}, \n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
         f"delivery_latency: {delivery_latency} us ,\n"
         f"num_slots: {num_slots}, \n"
         f"slot_duration: {slot_duration} us ,\n")
plt.title(title)
plt.tight_layout()
if scale == "log":
        plt.savefig(os.path.join(results_directory_experiment, "mean_latency_allUEs_log.png"))
elif scale == "linear":
        plt.savefig(os.path.join(results_directory_experiment, "mean_latency_allUEs_linear.png"))

plt.show()

slope = np.diff(mean_latencies_contention)/np.diff(lambda_range)
plt.yscale('log')
plt.plot(np.array(lambda_range[1:])*(schedule_contention.end_time - schedule_contention.start_time), slope)


In [ ]:
num_self_contention = []

for j in range(len(lambda_range)):
    self_contention_count = 0
    print("j", j)
    UE_temp = results_per_lambda_per_iteration_contention[lambda_range[j]][0]["UE0"]
    # print(UE_temp)
    latency_array = UE_temp.obtain_packet_latency()
    no_contention_array = []
    for i in range(len(latency_array)):
        if latency_array[i] == None:
            assert UE_temp.packets[i].arrival_time + 591.8 + 34 + 135 >= 791800, "Failed assertion" 
        elif np.floor(latency_array[i]- (591.8 + 34 + 135)) > 0:
            # pass
            
            if not (UE_temp.packets[i].arrival_time >= UE_temp.packets[i-1].arrival_time and \
                UE_temp.packets[i].arrival_time <= UE_temp.packets[i-1].delivery_time):

                print("i:", i)
                print(np.floor(latency_array[i]) - 591.8 - 34 - 135)
                print(UE_temp.packets[i])
                print("Previous packet")
                print(UE_temp.packets[i-1])
            self_contention_count += 1
            # assert UE_temp.packets[i].arrival_time >= UE_temp.packets[i-1].arrival_time and \
            #     UE_temp.packets[i].arrival_time <= UE_temp.packets[i-1].delivery_time, "Failed assertion"                
        else:
            no_contention_array.append(latency_array[i])
    print(np.mean(no_contention_array))
    print(np.mean([latency for latency in latency_array if latency is not None]))
    num_self_contention.append(self_contention_count/UE_temp.n_packets)
    print("self_contention_count", self_contention_count)   
    print(len(latency_array) - len(no_contention_array))

plt.plot(lambda_range*(schedule_contention.end_time - schedule_contention.start_time), num_self_contention)
plt.xlabel("lambda*schedule_duration")
plt.ylabel("Fraction of self-contention")
plt.title("Fraction of self-contention vs lambda")
plt.savefig(os.path.join(results_directory_experiment, "self_contention.png"))

plt.show()

In [ ]:
# Plotting inter-arrival times and arrival times
UE_temp = results_per_lambda_per_iteration_contention[lambda_range[0]][0]["UEs"]["UE0"]
arrival_times = []
for packet in UE_temp.packets:
    arrival_times.append(packet.arrival_time)

# plot histogram of arrival times
plt.hist(arrival_times, bins=100)
plt.xlabel("Arrival time (us)")
plt.ylabel("Frequency")
plt.title("Arrival time histogram")
plt.savefig(os.path.join(results_directory_experiment, "arrival_time_histogram.png"))
plt.show()

arrival_times = np.array(arrival_times)
inter_arrival_times = np.diff(arrival_times)
plt.hist(inter_arrival_times, bins=100)
plt.xlabel("Inter-arrival time (us)")
plt.ylabel("Frequency")
plt.title("Inter-arrival time histogram")
plt.savefig(os.path.join(results_directory_experiment, "inter_arrival_time_histogram.png"))
plt.show()

In [ ]:
print(end_time)

In [ ]:
for key in results_per_lambda:
    print("key", key)
    for key2 in results_per_lambda[key]:
        print("\tkey2", key2)
        for key3 in results_per_lambda[key][key2]:
            print("\t\tkey3", key3)
            print(results_per_lambda[key][key2][key3].packets[0])

In [ ]:
for key in results_per_lambda_contention:
    print("key", key)
    for key2 in results_per_lambda[key]:
        print("\tkey2", key2)
        for key3 in results_per_lambda_contention[key][key2]:
            print("\t\tkey3", key3)
            for key4 in results_per_lambda_contention[key][key2][key3]:
                print("\t\t\tkey4", key4)
            # print(results_per_lambda[key][key2][key3])

In [ ]:
np.log10(2000/(end_time - start_offset))
np.log10(1500/(end_time - start_offset))